{'faculties': [{'name': 'Computing', 'code': 'FCSC', 'departments': ['Computer Science and Software Engineering', 'Information Technology', 'Cyber Security', 'Interactive Media', 'Information Systems Engineering', 'Data Science']}], 'modules': [{'code': 'IT1010', 'name': 'Introduction to Computing', 'long_name': 'Introduction to Computing', 'description': 'Basic computing concepts'}, {'code': 'IT1020', 'name': 'IP', 'long_name': 'Introduction to Programming', 'description': 'Programming fundamentals'}, {'code': 'IT1030', 'name': 'MC', 'long_name': 'Mathematics for Computing', 'description': 'Mathematical concepts'}, {'code': 'IT1040', 'name': 'DBMS', 'long_name': 'Database Management Systems', 'description': 'Database fundamentals'}, {'code': 'IT1550', 'name': 'OOP', 'long_name': 'Object Oriented Programming', 'description': 'OOP concepts'}, {'code': 'IT1560', 'name': 'Web Development', 'long_name': 'Web Development', 'description': 'Web development basics'}, {'code': 'IT1570', 'name':

In [2]:
class Space:
    def __init__(self, *args):
        self.code = args[0]
        self.size = args[1]


class Group:
    def __init__(self, *args):
        self.id = args[0]
        self.size = args[1]


class Activity:
    def __init__(self, id, *args):
        self.id = id
        self.subject = args[0]
        self.teacher_id = args[1]
        self.group_ids = args[2]
        self.duration = args[3]


class Period:
    def __init__(self, *args):
        self.space = args[0]
        self.group = args[1]
        self.activity = args[2]

In [3]:
import json

# Load data from JSON file
with open('sliit_computing_dataset.json', 'r') as file:
    data = json.load(file)

# Create dictionaries to store instances
spaces_dict = {}
groups_dict = {}
activities_dict = {}
slots = []
# Populate the dictionaries with data from the JSON file
for space in data['spaces']:
    spaces_dict[space['code']] = Space(space['code'], space['capacity'])

for group in data['years']:
    groups_dict[group['id']] = Group(group['id'], group['size'])

for activity in data['activities']:
    activities_dict[activity['code']] = Activity(
        activity['code'], activity['subject'], activity['teacher_ids'][0], activity['subgroup_ids'], activity['duration'])

for day in ["MON", "TUE", "WED", "THU", "FRI"]:
    for id in range(1, 9):
        slots.append(day+str(id))
# Print the dictionaries to verify
print(spaces_dict)
print(groups_dict)
print(activities_dict)
print(slots)

{'LH401': <__main__.Space object at 0x000001AEA318A090>, 'LH501': <__main__.Space object at 0x000001AEA3142B40>, 'LAB501': <__main__.Space object at 0x000001AEA32B6ED0>, 'LAB502': <__main__.Space object at 0x000001AEA32B7350>}
{'Y1S1.1': <__main__.Group object at 0x000001AEA32B6E10>, 'Y1S1.2': <__main__.Group object at 0x000001AEA32B7380>, 'Y1S1.3': <__main__.Group object at 0x000001AEA32B6E40>, 'Y1S1.4': <__main__.Group object at 0x000001AEA32B6E70>, 'Y1S1.5': <__main__.Group object at 0x000001AEA32B73B0>, 'Y1S2.1': <__main__.Group object at 0x000001AEA32B7470>, 'Y1S2.2': <__main__.Group object at 0x000001AEA32B6EA0>, 'Y1S2.3': <__main__.Group object at 0x000001AEA32B6C90>, 'Y1S2.4': <__main__.Group object at 0x000001AEA32B6F30>, 'Y1S2.5': <__main__.Group object at 0x000001AEA32B73E0>, 'Y2S1.1': <__main__.Group object at 0x000001AEA32B74A0>, 'Y2S1.2': <__main__.Group object at 0x000001AEA32B6F90>, 'Y2S1.3': <__main__.Group object at 0x000001AEA32B7080>, 'Y2S1.4': <__main__.Group objec

In [ ]:
import random


def fitness(solution):
    room_overbooking = 0
    slot_conflicts = 0
    prof_conflicts = 0

    slot_assignments = {}
    prof_assignments = {}

    # Evaluate room overbooking and slot conflicts
    for entry in solution:
        course_id, student_group, room_id, valid_slots, professor = entry
        course = activities_dict[course_id]
        room = spaces_dict[room_id]

        # Room overbooking
        if room.size < get_classsize(course):
            room_overbooking += 1

        # Slot conflicts (same student in multiple classes at the same time)
        for slot_id in valid_slots:
            if slot_id not in slot_assignments:
                slot_assignments[slot_id] = []
            slot_assignments[slot_id].append((student_group, course_id))

        # Professor conflicts (same professor teaching multiple courses at the same time)
        if course.teacher_id not in prof_assignments:
            prof_assignments[course.teacher_id] = []
        prof_assignments[course.teacher_id].append(slot_id)

    # Calculate slot conflicts
    for slot_id, assignments in slot_assignments.items():
        student_groups_in_slot = {}
        for student_groups, course_id in assignments:
            for student_group in student_groups:
                if student_group in student_groups_in_slot:
                    slot_conflicts += 1  # Conflict when a student is assigned to two courses at the same time
                else:
                    student_groups_in_slot[student_group] = course_id

    # Calculate professor conflicts
    for professor, assigned_slots in prof_assignments.items():
        if len(assigned_slots) > len(set(assigned_slots)):  # Duplicate slots means conflict
            prof_conflicts += 1

    return room_overbooking, slot_conflicts, prof_conflicts


def get_classsize(activity: Activity) -> int:
    classsize = 0
    for id in activity.group_ids:
        classsize += groups_dict[id].size
    return classsize

# Updated create_population function


def create_population(pop_size):
    population = []
    for _ in range(pop_size):
        solution = []
        room_slot_assignments = {space.code: set()
                                 # Track slots per room
                                 for space in spaces_dict.values()}
        prof_slot_assignments = {prof: set() for prof in set(
            activity.teacher_id for activity in activities_dict.values())}  # Track professor slots

        for activity in activities_dict.values():
            # course = courses[course_id]

            student_groups = activity.group_ids
            rooms = [room for room in spaces_dict.values(
            ) if room.size >= get_classsize(activity)]
            # student_group = course.student  # Get student group
            selected_room = random.choice(rooms)  # Random room

            # Find consecutive slots for the course
            valid_slots = [
                slot for slot in slots if slot not in room_slot_assignments[selected_room.code] and slot not in prof_slot_assignments[activity.teacher_id]]

            if len(valid_slots) >= activity.duration:
                selected_slots = random.sample(valid_slots, activity.duration)
            else:
                selected_slots = random.sample(slots, activity.duration)

            for slot in selected_slots:
                room_slot_assignments[selected_room.code].add(slot)
                prof_slot_assignments[activity.teacher_id].add(slot)

            solution.append(
                (activity.id, student_groups, selected_room.code, selected_slots, activity.teacher_id))

        population.append(solution)
    return population

# Crossover function


def crossover(parent1, parent2):
    crossover_point = random.choice(range(1, len(parent1)))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2


# NSGA-II selection based on non-dominated sorting and crowding distance
def non_dominated_sorting(population, fitness_values):
    fronts = [[]]
    rank = [0] * len(population)
    domination_count = [0] * len(population)
    dominated_solutions = [[] for _ in range(len(population))]

    for p in range(len(population)):
        for q in range(len(population)):
            if dominates(fitness_values[p], fitness_values[q]):
                dominated_solutions[p].append(q)
            elif dominates(fitness_values[q], fitness_values[p]):
                domination_count[p] += 1

        if domination_count[p] == 0:
            rank[p] = 0
            fronts[0].append(p)

    front_counter = 0
    while len(fronts[front_counter]) > 0:
        next_front = []
        for p in fronts[front_counter]:
            for q in dominated_solutions[p]:
                domination_count[q] -= 1
                if domination_count[q] == 0:
                    rank[q] = front_counter + 1
                    next_front.append(q)
        front_counter += 1
        fronts.append(next_front)

    return fronts


# Crowding distance for maintaining diversity in the population
def crowding_distance(front, fitness_values):
    distance = [0] * len(front)
    if len(front) > 1:  # Only calculate crowding distance if there are at least two individuals in the front
        sorted_by_first = sorted(front, key=lambda x: fitness_values[x][0])
        sorted_by_second = sorted(front, key=lambda x: fitness_values[x][1])

        # Set infinity for boundary individuals
        distance[0] = distance[-1] = float('inf')
        for i in range(1, len(front) - 1):
            distance[i] = (fitness_values[sorted_by_first[i + 1]][0] - fitness_values[sorted_by_first[i - 1]][0]) + \
                          (fitness_values[sorted_by_second[i + 1]][1] -
                           fitness_values[sorted_by_second[i - 1]][1])

    return distance


# Tournament selection for parents
def tournament_selection(population, fitness_values):
    selected_parents = random.sample(list(range(len(population))), 2)
    p1, p2 = selected_parents[0], selected_parents[1]

    if dominates(fitness_values[p1], fitness_values[p2]):
        return p1
    else:
        return p2


def mutate(solution):
    mutation_point = random.randint(0, len(solution) - 1)
    course_id, student_group, _, _, _ = solution[mutation_point]

    new_room = random.choice(list(spaces_dict.values())).code  # Random room
    new_slot = random.choice(slots)  # Random slot

    professor = activities_dict[course_id].teacher_id
    # Update with new room and slot, keep course_id and student_group the same
    solution[mutation_point] = (
        course_id, student_group, new_room, new_slot, professor)
    return solution

# Domination check


def dominates(fit1, fit2):
    return (fit1[0] <= fit2[0] and fit1[1] <= fit2[1] and fit1[2] <= fit2[2]) and \
           (fit1[0] < fit2[0] or fit1[1] < fit2[1] or fit1[2] < fit2[2])


# Run NSGA-II algorithm
def run_nsga2(pop_size=50, generations=200):
    population = create_population(pop_size)
    for generation in range(generations):
        # Evaluate the fitness of the current population
        fitness_values = [fitness(solution) for solution in population]

        # Non-dominated sorting
        fronts = non_dominated_sorting(population, fitness_values)
        new_population = []

        # Iterate through each front and apply crossover and mutation
        for front in fronts:
            distances = crowding_distance(front, fitness_values)
            sorted_front = sorted(
                front, key=lambda x: distances[front.index(x)], reverse=True)

            # Select parents from the sorted front
            while len(new_population) < pop_size and len(sorted_front) > 1:
                parent1 = population[sorted_front[0]]
                parent2 = population[sorted_front[1]]

                # Crossover
                child1, child2 = crossover(parent1, parent2)

                # Mutation
                child1 = mutate(child1)
                child2 = mutate(child2)

                # Add children to the new population
                new_population.append(child1)
                new_population.append(child2)

        # Truncate to maintain population size
        population = new_population[:pop_size]

        # Print the progress
        if generation % 10 == 0:
            print(
                f"Generation {generation}: Population size {len(population)}")

    return population


# Running the algorithm
best_solution = run_nsga2()
print(f"Best solution: {best_solution}")

Generation 0: Population size 50
Generation 10: Population size 50
Generation 20: Population size 50
Generation 30: Population size 50
Generation 40: Population size 50
Generation 50: Population size 50
Generation 60: Population size 50
Generation 70: Population size 50
Generation 80: Population size 50
Generation 90: Population size 50
Generation 100: Population size 50
Generation 110: Population size 50
Generation 120: Population size 50
Generation 130: Population size 50
Generation 140: Population size 50
Generation 150: Population size 50
Generation 160: Population size 50
Generation 170: Population size 50
Generation 180: Population size 50
Generation 190: Population size 50
Best solution: [[('AC-001', ['Y1S1.1', 'Y1S1.2', 'Y1S1.3', 'Y1S1.4', 'Y1S1.5'], 'LH501', ['FRI4', 'WED5'], 'FA0000004'), ('AC-002', ['Y1S1.1'], 'LH501', 'MON3', 'FA0000001'), ('AC-003', ['Y1S1.2'], 'LH401', 'FRI5', 'FA0000006'), ('AC-004', ['Y1S1.3'], 'LAB502', 'THU1', 'FA0000009'), ('AC-005', ['Y1S1.4'], 'LAB

In [5]:
best_solution[0], fitness(best_solution[0])

([('AC-001',
   ['Y1S1.1', 'Y1S1.2', 'Y1S1.3', 'Y1S1.4', 'Y1S1.5'],
   'LH501',
   ['FRI4', 'WED5'],
   'FA0000004'),
  ('AC-002', ['Y1S1.1'], 'LH501', 'MON3', 'FA0000001'),
  ('AC-003', ['Y1S1.2'], 'LH401', 'FRI5', 'FA0000006'),
  ('AC-004', ['Y1S1.3'], 'LAB502', 'THU1', 'FA0000009'),
  ('AC-005', ['Y1S1.4'], 'LAB501', 'FRI3', 'FA0000008'),
  ('AC-006', ['Y1S1.5'], 'LH401', 'FRI6', 'FA0000001'),
  ('AC-007',
   ['Y1S1.1', 'Y1S1.2', 'Y1S1.3', 'Y1S1.4', 'Y1S1.5'],
   'LH401',
   ['MON4', 'FRI6'],
   'FA0000003'),
  ('AC-008', ['Y1S1.1'], 'LH401', ['THU5'], 'FA0000004'),
  ('AC-009', ['Y1S1.2'], 'LAB502', 'MON3', 'FA0000004'),
  ('AC-010', ['Y1S1.3'], 'LAB502', 'FRI7', 'FA0000005'),
  ('AC-011', ['Y1S1.4'], 'LAB502', 'MON5', 'FA0000004'),
  ('AC-012', ['Y1S1.5'], 'LAB502', 'MON5', 'FA0000006'),
  ('AC-013',
   ['Y1S1.1', 'Y1S1.2', 'Y1S1.3', 'Y1S1.4', 'Y1S1.5'],
   'LH401',
   ['THU8', 'THU3'],
   'FA0000006'),
  ('AC-014', ['Y1S1.1'], 'LH401', 'FRI6', 'FA0000001'),
  ('AC-015', ['Y1S1.2'

In [6]:
minimum_conflicts = 1000
final_solution = "hello"
for solution in best_solution:
    if minimum_conflicts >= sum(fitness(solution)):
        minimum_conflicts = sum(fitness(solution))
        final_solution = solution
print(minimum_conflicts)

10


In [ ]:
final_solution[0]

('AC-001',
 ['Y1S1.1', 'Y1S1.2', 'Y1S1.3', 'Y1S1.4', 'Y1S1.5'],
 'LH501',
 ['FRI4', 'WED5'],
 'FA0000004')

In [9]:
periods = []
for per in final_solution:
    for slot in per[3]:
        periods.append(Period(per[2],slot,activities_dict[per[0]]))
fitness(final_solution)
periods

In [ ]:
Activity(final_solution)